In [1]:
import pandas as pd
import numpy as np
import nltk
import pickle
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import ast

In [3]:
# initialize dataframe
df = pd.read_csv('JP_alldata 0802.csv')

df['public_metrics'][0]

"{'retweet_count': 76, 'reply_count': 5, 'like_count': 311, 'quote_count': 6}"

In [4]:
def res(string_dict):
    return ast.literal_eval(string_dict)

# set the dictinary as a dictionary
df['public_metrics'] = df['public_metrics'].apply(res)

In [5]:
df.columns

Index(['Unnamed: 0', 'created_at', 'text', 'entities', 'public_metrics',
       'union name', 'Relevance:blank = relevant, 1=irrelevant, 2=discuss'],
      dtype='object')

In [6]:
# obtain the columns
df = df[['created_at','text', 'entities','public_metrics','union name']]

In [7]:
df

,created_at,text,entities,public_metrics,union name
0,2022-06-08T01:39:00.000Z,"Every student in America—Black, brown or white...","{'annotations': [{'start': 17, 'end': 23, 'pro...","{'retweet_count': 76, 'reply_count': 5, 'like_...",NEAToday
1,2022-05-20T00:10:00.000Z,"So far, education advocates have not had a cle...","{'urls': [{'start': 109, 'end': 132, 'url': 'h...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",NEAToday
2,2022-05-19T17:07:04.000Z,When educators and management come together us...,"{'urls': [{'start': 164, 'end': 187, 'url': 'h...","{'retweet_count': 6, 'reply_count': 1, 'like_c...",NEAToday
3,2022-05-09T17:47:01.000Z,The mental health of our students must be a to...,"{'annotations': [{'start': 202, 'end': 227, 'p...","{'retweet_count': 15, 'reply_count': 4, 'like_...",NEAToday
4,2022-04-22T19:46:02.000Z,One way to honor #EarthDay: Introduce your stu...,"{'hashtags': [{'start': 17, 'end': 26, 'tag': ...","{'retweet_count': 15, 'reply_count': 2, 'like_...",NEAToday
...,...,...,...,...,...
12135,2022-08-16T20:40:18.000Z,The #InflationReductionAct:\r\n💊 Lowers prescr...,"{'urls': [{'start': 261, 'end': 284, 'url': 'h...","{'retweet_count': 8, 'reply_count': 0, 'like_c...",AFSCME
12136,2022-08-16T20:15:08.000Z,"""The Inflation Reduction Act is an all-around ...","{'urls': [{'start': 253, 'end': 276, 'url': 'h...","{'retweet_count': 10, 'reply_count': 2, 'like_...",IBEW
12137,2022-08-16T01:54:32.000Z,Mining vehicle operators represented by #Teams...,"{'urls': [{'start': 248, 'end': 271, 'url': 'h...","{'retweet_count': 3, 'reply_count': 3, 'like_c...",Teamsters
12138,2022-08-15T14:48:29.000Z,"""The fastest growing industries - tech and gre...","{'mentions': [{'start': 124, 'end': 131, 'user...","{'retweet_count': 9, 'reply_count': 1, 'like_c...",APWUnational


In [8]:
#often, we want to remove stopwords
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 
                     'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 
                     'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
                     'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
                     'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 
                     'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
                     'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 
                     'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
                     'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 
                     'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 
                     'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 'can', 'will',
                     'just', 'dont', 'should', 'aint', 'arent', 'couldn', 'could', 'would', 'much', 'must',
                     'didnt', 'doesnt', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shan',
                     'shouldnt', 'wasnt', 'werent', 'wont', 'wouldnt','rt','mt','amp','mil','ve','don','s','st','pres','req','aft','ua', 'via']

In [9]:
 #adopted from catriscode, https://catriscode.com/2021/05/01/tweets-cleaning-with-python/

import numpy as np
import re

def clean_tweet(tweet):
    raw_tweet = str(tweet)
    if type(tweet) == float:
        return ""
    temp = re.sub("'", "", str(tweet)) # to avoid removing contractions in english
    temp = temp.lower()
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub(r"www.\S+", "", temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    #temp = re.sub("[^a-z0-9]"," ", temp) # this removes all letters including CO2
    temp = re.sub("[^a-z]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    
    if temp is None:
        print (raw_tweet,temp)
        
    return temp

In [10]:
# create clean text column
df['clean_text'] = df['text'].apply(clean_tweet)

In [11]:
df = df.dropna(subset=['clean_text'])
df
#empty data with cleaning procedure

#`reset_index()`will recreate index column every time we run it on same data 
#`drop = True` paramater won’t create that as column in the dataframe, look at the difference between following two dataset
df = df.reset_index(drop=False) 

In [12]:
df

,index,created_at,text,entities,public_metrics,union name,clean_text
0,0,2022-06-08T01:39:00.000Z,"Every student in America—Black, brown or white...","{'annotations': [{'start': 17, 'end': 23, 'pro...","{'retweet_count': 76, 'reply_count': 5, 'like_...",NEAToday,every student america black brown white native...
1,1,2022-05-20T00:10:00.000Z,"So far, education advocates have not had a cle...","{'urls': [{'start': 109, 'end': 132, 'url': 'h...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",NEAToday,far education advocates clear role addressing ...
2,2,2022-05-19T17:07:04.000Z,When educators and management come together us...,"{'urls': [{'start': 164, 'end': 187, 'url': 'h...","{'retweet_count': 6, 'reply_count': 1, 'like_c...",NEAToday,educators management come together using susta...
3,3,2022-05-09T17:47:01.000Z,The mental health of our students must be a to...,"{'annotations': [{'start': 202, 'end': 227, 'p...","{'retweet_count': 15, 'reply_count': 4, 'like_...",NEAToday,mental health students top priority school com...
4,4,2022-04-22T19:46:02.000Z,One way to honor #EarthDay: Introduce your stu...,"{'hashtags': [{'start': 17, 'end': 26, 'tag': ...","{'retweet_count': 15, 'reply_count': 2, 'like_...",NEAToday,one way honor introduce students girls green w...
...,...,...,...,...,...,...,...
12135,12135,2022-08-16T20:40:18.000Z,The #InflationReductionAct:\r\n💊 Lowers prescr...,"{'urls': [{'start': 261, 'end': 284, 'url': 'h...","{'retweet_count': 8, 'reply_count': 0, 'like_c...",AFSCME,lowers prescription drug health care costs inv...
12136,12136,2022-08-16T20:15:08.000Z,"""The Inflation Reduction Act is an all-around ...","{'urls': [{'start': 253, 'end': 276, 'url': 'h...","{'retweet_count': 10, 'reply_count': 2, 'like_...",IBEW,inflation reduction act around victory climate...
12137,12137,2022-08-16T01:54:32.000Z,Mining vehicle operators represented by #Teams...,"{'urls': [{'start': 248, 'end': 271, 'url': 'h...","{'retweet_count': 3, 'reply_count': 3, 'like_c...",Teamsters,mining vehicle operators represented local ari...
12138,12138,2022-08-15T14:48:29.000Z,"""The fastest growing industries - tech and gre...","{'mentions': [{'start': 124, 'end': 131, 'user...","{'retweet_count': 9, 'reply_count': 1, 'like_c...",APWUnational,fastest growing industries tech green energy m...


In [13]:
# create a list of cleaned text
df['clean_text_list'] = [l.split() for l in df['clean_text']]

In [14]:
df

,index,created_at,text,entities,public_metrics,union name,clean_text,clean_text_list
0,0,2022-06-08T01:39:00.000Z,"Every student in America—Black, brown or white...","{'annotations': [{'start': 17, 'end': 23, 'pro...","{'retweet_count': 76, 'reply_count': 5, 'like_...",NEAToday,every student america black brown white native...,"[every, student, america, black, brown, white,..."
1,1,2022-05-20T00:10:00.000Z,"So far, education advocates have not had a cle...","{'urls': [{'start': 109, 'end': 132, 'url': 'h...","{'retweet_count': 3, 'reply_count': 1, 'like_c...",NEAToday,far education advocates clear role addressing ...,"[far, education, advocates, clear, role, addre..."
2,2,2022-05-19T17:07:04.000Z,When educators and management come together us...,"{'urls': [{'start': 164, 'end': 187, 'url': 'h...","{'retweet_count': 6, 'reply_count': 1, 'like_c...",NEAToday,educators management come together using susta...,"[educators, management, come, together, using,..."
3,3,2022-05-09T17:47:01.000Z,The mental health of our students must be a to...,"{'annotations': [{'start': 202, 'end': 227, 'p...","{'retweet_count': 15, 'reply_count': 4, 'like_...",NEAToday,mental health students top priority school com...,"[mental, health, students, top, priority, scho..."
4,4,2022-04-22T19:46:02.000Z,One way to honor #EarthDay: Introduce your stu...,"{'hashtags': [{'start': 17, 'end': 26, 'tag': ...","{'retweet_count': 15, 'reply_count': 2, 'like_...",NEAToday,one way honor introduce students girls green w...,"[one, way, honor, introduce, students, girls, ..."
...,...,...,...,...,...,...,...,...
12135,12135,2022-08-16T20:40:18.000Z,The #InflationReductionAct:\r\n💊 Lowers prescr...,"{'urls': [{'start': 261, 'end': 284, 'url': 'h...","{'retweet_count': 8, 'reply_count': 0, 'like_c...",AFSCME,lowers prescription drug health care costs inv...,"[lowers, prescription, drug, health, care, cos..."
12136,12136,2022-08-16T20:15:08.000Z,"""The Inflation Reduction Act is an all-around ...","{'urls': [{'start': 253, 'end': 276, 'url': 'h...","{'retweet_count': 10, 'reply_count': 2, 'like_...",IBEW,inflation reduction act around victory climate...,"[inflation, reduction, act, around, victory, c..."
12137,12137,2022-08-16T01:54:32.000Z,Mining vehicle operators represented by #Teams...,"{'urls': [{'start': 248, 'end': 271, 'url': 'h...","{'retweet_count': 3, 'reply_count': 3, 'like_c...",Teamsters,mining vehicle operators represented local ari...,"[mining, vehicle, operators, represented, loca..."
12138,12138,2022-08-15T14:48:29.000Z,"""The fastest growing industries - tech and gre...","{'mentions': [{'start': 124, 'end': 131, 'user...","{'retweet_count': 9, 'reply_count': 1, 'like_c...",APWUnational,fastest growing industries tech green energy m...,"[fastest, growing, industries, tech, green, en..."


In [15]:
# save dataframe
df.to_csv('All Tweets Twitter Cleaned Text.csv')

In [16]:
df = pd.read_csv('All Tweets Twitter Cleaned Text.csv')

In [17]:
def res(string_dict):
    return ast.literal_eval(string_dict)

# set the dictinary as a dictionary
df['clean_text_list'] = df['clean_text_list'].apply(res)

In [18]:
# initialize words
words = df['clean_text_list']

words

0        [every, student, america, black, brown, white,...
1        [far, education, advocates, clear, role, addre...
2        [educators, management, come, together, using,...
3        [mental, health, students, top, priority, scho...
4        [one, way, honor, introduce, students, girls, ...
                               ...                        
12135    [lowers, prescription, drug, health, care, cos...
12136    [inflation, reduction, act, around, victory, c...
12137    [mining, vehicle, operators, represented, loca...
12138    [fastest, growing, industries, tech, green, en...
12139    [happy, th, birthday, social, security, now, t...
Name: clean_text_list, Length: 12140, dtype: object

In [19]:
import pandas as pd

# Create a sample Series with lists
my_series = pd.Series([['read', 're'], ['reply', 'care'], ['respect', 're']])

# Define the remove_re function to remove a specified bad string from all lists
def remove_string(lst, bad_string):
    return [s for s in lst if s != bad_string]

def remove_bad_string(series, bad_string):
    return series.apply(lambda lst: remove_string(lst, bad_string))

# Apply the remove_bad_string function to the Series
my_series = remove_bad_string(my_series, 're')

# Print the modified Series
print(my_series)

words = remove_bad_string(words, 're')
words = remove_bad_string(words, 'll')
words = remove_bad_string(words, 't')
words = remove_bad_string(words, 'b')
words = remove_bad_string(words, 'yrs')
words = remove_bad_string(words, 'w')
words = remove_bad_string(words, 'u')
words = remove_bad_string(words, 'wi')

0           [read]
1    [reply, care]
2        [respect]
dtype: object


In [20]:
type(words)

pandas.core.series.Series

In [21]:
words[0]

['every',
 'student',
 'america',
 'black',
 'brown',
 'white',
 'native',
 'newcomer',
 'deserves',
 'welcoming',
 'safe',
 'learning',
 'environment',
 'free',
 'fear',
 'gun',
 'violence',
 'school',
 'community']

In [22]:
import gensim
from gensim import corpora
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

dictionary = corpora.Dictionary(words)
corpus = [dictionary.doc2bow(word) for word in words]

# define the number of topics to identify
num_topics = 7


# build the LDA model
lda_model = gensim.models.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                   num_topics=num_topics,
                                   random_state=100,
                                   update_every=1,
                                   chunksize=10,
                                   passes=10,
                                   alpha='auto',
                                   per_word_topics=True)

# visualize the topics using pyLDAvis
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)

# print the top words for each topic
for topic in lda_model.print_topics():
    print(topic)

E:\Anaconda\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


(0, '0.094*"union" + 0.047*"environment" + 0.043*"program" + 0.033*"safe" + 0.023*"costs" + 0.019*"say" + 0.018*"every" + 0.017*"community" + 0.017*"know" + 0.016*"america"')
(1, '0.034*"energy" + 0.031*"climate" + 0.024*"jobs" + 0.022*"local" + 0.019*"sustainable" + 0.017*"today" + 0.015*"change" + 0.014*"good" + 0.014*"care" + 0.013*"us"')
(2, '0.050*"work" + 0.040*"water" + 0.033*"people" + 0.022*"clean" + 0.016*"year" + 0.014*"zero" + 0.014*"live" + 0.012*"worker" + 0.011*"school" + 0.011*"american"')
(3, '0.056*"safety" + 0.034*"make" + 0.025*"president" + 0.022*"public" + 0.021*"read" + 0.019*"unions" + 0.016*"many" + 0.016*"choice" + 0.015*"fight" + 0.013*"movement"')
(4, '0.056*"green" + 0.045*"new" + 0.044*"health" + 0.034*"food" + 0.029*"working" + 0.021*"one" + 0.021*"future" + 0.018*"crisis" + 0.017*"communities" + 0.013*"create"')
(5, '0.053*"now" + 0.041*"learn" + 0.036*"support" + 0.032*"action" + 0.022*"leaders" + 0.020*"programs" + 0.018*"creating" + 0.018*"years" + 0.

In [23]:
from gensim.models import CoherenceModel

# Analyzing the model

In [24]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.498748692839802

Coherence Score:  0.24682900545072983


for number_topics in range(4,11):
    
    print('number of topics: ',number_topics)

    lda_model = gensim.models.LdaModel(corpus=corpus,
                                       id2word=dictionary,
                                       num_topics=number_topics,
                                       random_state=100,
                                       update_every=1,
                                       chunksize=10,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

    # Compute Perplexity
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
    # a measure of how good the model is. lower the better.


    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=words, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)

In [25]:
# num topics of 7 is best

In [26]:
pyLDAvis.display(vis_data)

(0, '0.094*"union" + 0.047*"environment" + 0.043*"program" + 0.033*"safe" + 0.023*"costs" + 0.019*"say" + 0.018*"every" + 0.017*"community" + 0.017*"know" + 0.016*"america"')
(1, '0.034*"energy" + 0.031*"climate" + 0.024*"jobs" + 0.022*"local" + 0.019*"sustainable" + 0.017*"today" + 0.015*"change" + 0.014*"good" + 0.014*"care" + 0.013*"us"')
(2, '0.050*"work" + 0.040*"water" + 0.033*"people" + 0.022*"clean" + 0.016*"year" + 0.014*"zero" + 0.014*"live" + 0.012*"worker" + 0.011*"school" + 0.011*"american"')
(3, '0.056*"safety" + 0.034*"make" + 0.025*"president" + 0.022*"public" + 0.021*"read" + 0.019*"unions" + 0.016*"many" + 0.016*"choice" + 0.015*"fight" + 0.013*"movement"')
(4, '0.056*"green" + 0.045*"new" + 0.044*"health" + 0.034*"food" + 0.029*"working" + 0.021*"one" + 0.021*"future" + 0.018*"crisis" + 0.017*"communities" + 0.013*"create"')
(5, '0.053*"now" + 0.041*"learn" + 0.036*"support" + 0.032*"action" + 0.022*"leaders" + 0.020*"programs" + 0.018*"creating" + 0.018*"years" + 0.018*"paid" + 0.017*"board"')
(6, '0.068*"workers" + 0.028*"members" + 0.024*"hurricane" + 0.019*"need" + 0.019*"heat" + 0.017*"help" + 0.015*"coal" + 0.014*"disaster" + 0.013*"relief" + 0.012*"act"'

topic 0 seems to be related to labor unions and workplace safety, while topic 1 is focused on climate change, sustainable energy, and local jobs. Topic 2 seems to be related to clean water and people's access to it, and topic 3 is related to public safety, the role of unions, and political movements. Topic 4 is related to environmental and public health, food, and the future, while topic 5 is related to taking action and supporting leaders in environmental and social issues. Finally, topic 6 seems to be related to workers' rights during natural disasters, coal, and relief efforts.

## Analysis

Topic 0: This topic seems to be related to coal mining and energy production. Words like "coal", "miners", "energy", and "water" suggest that the discussion may be about the environmental impact of coal mining and the need for alternative sources of energy.

Topic 1: This topic seems to be related to sustainability and environmental justice. Words like "sustainability", "environmental", "justice", and "state" suggest that the discussion may be about promoting sustainable practices and addressing environmental inequalities.

Topic 2: This topic seems to be related to sustainable jobs and climate change. Words like "sustainable", "jobs", "climate", and "need" suggest that the discussion may be about creating job opportunities that are both sustainable and contribute to addressing climate change.

Topic 3: This topic seems to be related to the impact of climate change on workers and unions. Words like "union", "members", "hurricane", and "power" suggest that the discussion may be about the need for unions to address the impact of climate change on workers and their communities.

Topic 4: This topic seems to be related to supporting programs that protect workers. Words like "program", "housekeepers", "support", and "protect" suggest that the discussion may be about advocating for programs that support and protect workers, particularly those in low-wage jobs.

Topic 5: This topic seems to be related to labor and food justice. Words like "food", "labor", "job", and "strike" suggest that the discussion may be about promoting fair labor practices in the food industry and advocating for food justice.

Topic 6: This topic seems to be related to promoting a green economy and workplace safety. Words like "green", "safety", "health", and "unions" suggest that the discussion may be about promoting a sustainable and safe workplace environment.




